Define Basic Parameter

In [1]:
import requests
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import json
import pandas as pd
from collections import defaultdict
from datetime import datetime
import matplotlib.pyplot as plt
import numpy as np



ModuleNotFoundError: No module named 'plotly'

In [ ]:
# Enter the latitude and longitude for the location you want to query.
# The script will use these coordinates to fetch the corresponding Micro-Watershed (MWS) ID,
# and then retrieve the data associated with that specific microwatershed.

longitude = "84.8148139979161"
latitude  = "21.061542953125095"

#  Base URL for the CoreStack  API end point
base_url = 'https://geoserver.core-stack.org/api/v1/'

# Endpoint to retrieve the Micro-Watershed (MWS) ID using latitude and longitude
lat_long_mws_api_endpoint = 'get_mwsid_by_latlon/'

# Endpoint to retrieve detailed data for a given Micro-Watershed (MWS) ID
geojson_api_endpoint = 'get_generated_layer_urls/'

# Add your API key here.
# You can generate an API key by logging into https://dashboard.core-stack.org with an Organization Admin account.
# Only Org Admins have permission to generate API keys.
# If you are not an Org Admin, please contact your admin to obtain access.
Api_key = 'rBKKRCKl.jsfeUjArYxscYFJZf4FY0L7NuddaSp5u'

output_file = "deltaG_fortnight_anugul_anugul.geojson"

In [ ]:
# Define the HTTP request headers with the API key
headers = {
    "X-API-Key": Api_key
}

# Define the query parameters to be sent in the API request

params = {
    "latitude": latitude,
    "longitude": longitude
}

# Construct the full API URL by combining base URL and endpoint
mws_lat_long_api_url = f"{base_url}{lat_long_mws_api_endpoint}"


# Make the GET request with headers and parameters
response = requests.get(mws_lat_long_api_url, params=params, headers=headers)

# Check if the request was successful (status code 200)
if response.status_code == 200:
    # Parse JSON response data
    response_data_mws_lat_long = response.json()
    # You can now work with 'response_data_mws_lat_long'
    print("API call successful. Data received.")
else:
    # Handle errors - print status code and response text for debugging
    print(f"API call failed with status code: {response.status_code}")
    print("Response content:", response.text)
    # Optional: you can raise an exception or handle retries here





API call successful. Data received.


In [ ]:
def build_query_params_from_lat_long_api(mws_lat_long_response):
    """
    Extracts relevant query parameters from the MWS lat-long API response
    and returns them in a dictionary format suitable for further API calls.

    Args:
        mws_lat_long_response (dict): Response JSON from the lat-long API containing location info.

    Returns:
        dict: A dictionary with keys 'state', 'district', 'tehsil', and 'mws_id'.
    """
    d = {}

    # Extract state name from the API response
    d['state'] = mws_lat_long_response['State']

    # Extract district name from the API response
    d['district'] = mws_lat_long_response['District']

    # Extract tehsil (sub-district) name from the API response
    d['tehsil'] = mws_lat_long_response['Tehsil']



    return d
print(response_data_mws_lat_long)

{'State': 'ODISHA', 'District': 'ANUGUL', 'Tehsil': 'ANUGUL', 'uid': '7_21174'}


In [ ]:
# Build query parameters from the previous API response
param2 = build_query_params_from_lat_long_api(response_data_mws_lat_long)

# Construct the full URL for the MWS data API endpoint
mws_data_url = f"{base_url}{geojson_api_endpoint}"

# Make the GET request with the constructed parameters and headers
response = requests.get(mws_data_url, params=param2, headers=headers)

# Check if the request was successful
if response.status_code == 200:
    # Parse the JSON response and extract the hydrological annual data
    hydrological_data = response.json()
    hydrology_layers = [ layer for layer in hydrological_data if "deltaG" in layer.get("layer_url", "")]
    geojson_url  = hydrology_layers[0]['layer_url']
    print (geojson_url)

else:
    # Print error information if the request failed
    print(f"Failed to retrieve data, status code: {response.status_code}")
    print("Response content:", response.text)



https://geoserver.core-stack.org:8443/geoserver/mws_layers/ows?service=WFS&version=1.0.0&request=GetFeature&typeName=mws_layers:deltaG_well_depth_anugul_anugul&outputFormat=application/json


In [ ]:
# Send an HTTP GET request to the URL
# verify=False disables SSL certificate verification (only use this if absolutely necessary)
response = requests.get(geojson_url, verify=False)

# Check if the request was successful (HTTP status code 200 means OK)
if response.status_code == 200:
    # Open a local file in binary write mode and write the downloaded content to it
    with open(output_file, "wb") as f:
        f.write(response.content)

    # Confirm that the file was saved successfully
    print(f"File saved as {output_file}")
else:
    # Print an error message if the request failed
    print(f"Failed to download: HTTP {response.status_code}")

/usr/local/lib/python3.12/dist-packages/urllib3/connectionpool.py:1097: InsecureRequestWarning:

Unverified HTTPS request is being made to host 'geoserver.core-stack.org'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings



File saved as deltaG_fortnight_anugul_anugul.geojson


In [ ]:
def GetWaterBalanceTimeSeriesFromGeojson(geojson_filepath):
    """
    Loads yearly water balance data from a GeoJSON file.

    Args:
        geojson_filepath (str): Path to the input GeoJSON file.

    Returns:
        list of dict: Each dict contains year and water balance components
                      (DeltaG, ET, Precipitation, RunOff, G, WellDepth).
    """

    import json
    from datetime import datetime

    # Load geojson
    with open(geojson_filepath, "r") as f:
        feature_collection = json.load(f)

    data = feature_collection["features"][0]["properties"]

    timeseries_data = []

    for key, value in data.items():

        # Accept only keys like "2017_2018"
        if not (isinstance(key, str) and key.replace("_", "").isdigit() and "_" in key):
            continue

        # Parse the JSON string
        if isinstance(value, str):
            values = json.loads(value)
        else:
            continue  # skip numeric values such as Net2017_22

        # Convert "2017_2018" → year as int (first year)
        year_start = int(key.split("_")[0])
        date = datetime(year_start, 1, 1)

        timeseries_data.append({
            "date": date,
            "year_str": key,   # keep original label for plotting if needed
            "DeltaG": values.get("DeltaG", 0.0),
            "ET": values.get("ET", 0.0),
            "Precipitation": values.get("Precipitation", 0.0),
            "RunOff": values.get("RunOff", 0.0),
            "G": values.get("G", 0.0),
            "WellDepth": values.get("WellDepth", None)
        })

    # Sort chronologically
    timeseries_data.sort(key=lambda x: x["date"])

    return timeseries_data


In [ ]:
timeseries_data = GetWaterBalanceTimeSeriesFromGeojson(output_file)
raw_data = timeseries_data
print(type(raw_data))   # should be <class 'list'>

# -----------------------------------------
# Convert list of dicts into DataFrame
# -----------------------------------------
df = pd.DataFrame(raw_data)

# Convert "date" column to datetime
df["date"] = pd.to_datetime(df["date"])

# Extract year for slider (first year of range)
df["Year"] = df["date"].dt.year

display(df)

# Unique years for slider
unique_years = sorted(df["Year"].unique())

# -----------------------------------------
# Create Subplots
# -----------------------------------------
fig = make_subplots(
    rows=3, cols=1, shared_xaxes=True,
    subplot_titles=("Precipitation and RunOff",
                    "Evapotranspiration (ET)",
                    "DeltaG")
)

# --- 1) Precipitation (Bars) ---
fig.add_trace(
    go.Bar(
        x=df['date'], y=df['Precipitation'],
        name="Precipitation (mm)",
        marker_color="blue",
        opacity=0.6
    ),
    row=1, col=1
)

# --- 1b) Runoff (Line + Markers) ---
fig.add_trace(
    go.Scatter(
        x=df['date'], y=df['RunOff'],
        mode='lines+markers',
        name="RunOff (mm)",
        line=dict(color="magenta")
    ),
    row=1, col=1
)

# --- 2) ET ---
fig.add_trace(
    go.Scatter(
        x=df['date'], y=df['ET'],
        mode='lines+markers',
        name="ET",
        line=dict(color="green"),
        fill='tozeroy'
    ),
    row=2, col=1
)

# --- 3) DeltaG ---
max_abs_deltaG = df['DeltaG'].abs().max()

fig.add_trace(
    go.Scatter(
        x=df['date'], y=df['DeltaG'],
        mode='lines',
        name="DeltaG",
        line=dict(color="orange")
    ),
    row=3, col=1
)

# -----------------------------------------
# Slider (by Year)

# -----------------------------------------
# Layout
# -----------------------------------------
fig.update_layout(
    height=900,
    title=dict(
        text="Water Balance Components Over Time",
        x=0.5, xanchor="center", y=0.95
    ),
    sliders=sliders,
    legend=dict(
        orientation="h",
        y=0.92,
        x=0.5, xanchor="center"
    )
)

# Axis labels and styling
fig.update_yaxes(title_text="mm", row=1, col=1)
fig.update_yaxes(title_text="mm", row=2, col=1)
fig.update_yaxes(
    title_text="DeltaG", row=3, col=1,
    range=[-max_abs_deltaG, max_abs_deltaG],
    zeroline=True, zerolinewidth=2, zerolinecolor='gray'
)
fig.update_xaxes(title_text="Date", row=3, col=1)

fig.show()


<class 'list'>


,date,year_str,DeltaG,ET,Precipitation,RunOff,G,WellDepth,Year
0,2017-01-01,2017_2018,236.208926,956.976034,1380.022937,186.837977,236.208926,7.873631,2017
1,2018-01-01,2018_2019,255.353529,1000.926109,1693.870334,437.590697,491.562455,8.511784,2018
2,2019-01-01,2019_2020,262.163093,998.094047,1639.828059,379.570919,753.725548,8.738770,2019
3,2020-01-01,2020_2021,169.912953,903.999426,1349.900366,275.987986,923.638501,5.663765,2020
4,2021-01-01,2021_2022,59.030338,892.710255,1142.575724,190.835131,982.668839,1.967678,2021
5,2022-01-01,2022_2023,92.529333,897.457398,1192.465235,202.478504,1075.198172,3.084311,2022
6,2023-01-01,2023_2024,60.438580,848.805423,1125.114684,215.870681,1135.636752,2.014619,2023
7,2024-01-01,2024_2025,188.255147,822.584755,1188.536472,177.696571,1323.891899,6.275172,2024
